In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2

from keras.preprocessing.image import ImageDataGenerator

In [2]:
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-4
N_CHANNELS = 1

In [3]:
base_dir = './content/DataSet2/Denoising'

gaussian_dir = os.path.join(base_dir, 'Gaussian')
gaussian_train_dir = os.path.join(gaussian_dir, 'Train')
gaussian_test_dir = os.path.join(gaussian_dir, 'Test')
gaussian_val_dir = os.path.join(gaussian_dir, 'Validation')

periodic_dir = os.path.join(base_dir, 'Periodic')
Periodic_train_dir = os.path.join(periodic_dir, 'Train')
Periodic_test_dir = os.path.join(periodic_dir, 'Test')
Periodic_val_dir = os.path.join(periodic_dir, 'Validation')

salt_dir = os.path.join(base_dir, 'Salt')
Salt_train_dir = os.path.join(salt_dir, 'Train')
Salt_test_dir = os.path.join(salt_dir, 'Test')
Salt_val_dir = os.path.join(salt_dir, 'Validation')

In [4]:
data = pd.read_excel('./content/Labels.xlsx')
data.head()

,Denoise Image,Noise Type,Noisy Image
0,Without-Noise/cast_def_0_0.jpeg,Gaussian,With-Noise/cast_def_0_0.jpeg
1,Without-Noise/cast_def_0_2.jpeg,Gaussian,With-Noise/cast_def_0_2.jpeg
2,Without-Noise/cast_def_0_22.jpeg,Gaussian,With-Noise/cast_def_0_22.jpeg
3,Without-Noise/cast_def_0_25.jpeg,Gaussian,With-Noise/cast_def_0_25.jpeg
4,Without-Noise/cast_def_0_33.jpeg,Gaussian,With-Noise/cast_def_0_33.jpeg


In [5]:
def read_images(folder_path):
    arr = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            if(N_CHANNELS==1):
                arr.append(cv2.imread(file_path,cv2.IMREAD_GRAYSCALE))
            else:
                arr.append(cv2.imread(file_path))
    return np.array(arr)

In [6]:
X_train = read_images(os.path.join(Periodic_train_dir, 'With-Noise'))
y_train = read_images(os.path.join(Periodic_train_dir, 'Without-Noise'))

X_test = read_images(os.path.join(Periodic_test_dir, 'With-Noise'))
y_test = read_images(os.path.join(Periodic_test_dir, 'Without-Noise'))

# X_val = read_images(os.path.join(Periodic_val_dir, 'With-Noise'))
# y_val = read_images(os.path.join(Periodic_val_dir, 'Without-Noise'))

## Models

In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Define callbacks for early stopping, model checkpointing, and learning rate reduction
my_callbacks = [
    EarlyStopping(monitor='val_loss', patience=5),
    ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3)
]

In [13]:
import keras

def get_model(img_size, num_classes):

    inputs = keras.Input(shape=img_size+(1,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = keras.layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = keras.layers.Activation("relu")(x)
        x = keras.layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Activation("relu")(x)
        x = keras.layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = keras.layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = keras.layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = keras.layers.Activation("relu")(x)
        x = keras.layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Activation("relu")(x)
        x = keras.layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.UpSampling2D(2)(x)

        # Project residual
        residual = keras.layers.UpSampling2D(2)(previous_block_activation)
        residual = keras.layers.Conv2D(filters, 1, padding="same")(residual)
        x = keras.layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = keras.layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model

model = get_model((IMAGE_HEIGHT,IMAGE_WIDTH), 256)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_17 (Conv2D)             (None, 128, 128, 32  320         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_30 (BatchN  (None, 128, 128, 32  128        ['conv2d_17[0][0]']              
 ormalization)                  )                                                           

In [14]:
history = 0
with tf.device('/GPU:0'):
    history = model.fit(
        X_train, 
        y_train,
        epochs=5,
        callbacks=my_callbacks,
        # validation_data=(np.array(X_val), np.array(y_val)),
    )

Epoch 1/5


ResourceExhaustedError: Graph execution error:

Detected at node 'model_2/up_sampling2d_18/resize/ResizeNearestNeighbor' defined at (most recent call last):
    File "d:\Programming\Anaconda3\envs\tf\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "d:\Programming\Anaconda3\envs\tf\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "d:\Programming\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "d:\Programming\Anaconda3\envs\tf\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "d:\Programming\Anaconda3\envs\tf\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "d:\Programming\Anaconda3\envs\tf\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "d:\Programming\Anaconda

In [8]:
pd.DataFrame(history.history)[['accuracy','val_accuracy']].plot()
plt.title("Accuracy")
plt.show()

pd.DataFrame(history.history)[['loss','val_loss']].plot()
plt.title("Loss")
plt.show()

NameError: name 'history' is not defined

In [ ]:
predict = autoencoder.predict(X_test[0])
cv2.PSNR(predict, y_test[0])

## Unet++

In [8]:
from tensorflow.keras.layers import Input, Conv2D, Dense, UpSampling2D, Concatenate, Activation, MaxPooling2D
from tensorflow.keras.models import Model



# Input layer
noisy_inputs = Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS))

# Encoder: Adjust depth and filters based on noise complexity
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(noisy_inputs)
conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv3)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv6)

# Nested encoders (optional)
# ... consider adding more encoders with larger filter sizes

# Decoder (ensuring shape compatibility for concatenation)
up1 = UpSampling2D(size=(2, 2))(pool3)
up1 = Conv2D(128, (1, 1), activation='relu')(up1)  # Match channels for concatenation

concat1 = Concatenate()([up1, conv5])  # Shapes will now match
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(concat1)
conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

up2 = UpSampling2D(size=(2, 2))(conv8)  # Corrected argument
up2 = Conv2D(64, (1, 1), activation='relu')(up2)  # Adjust filter size as needed

concat2 = Concatenate()([up2, conv4])  # Shapes will now match (64,...)
conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat2)
conv10 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)

up3 = UpSampling2D(size=(2, 2))(conv10)  # Corrected argument
up3 = Conv2D(32, (1, 1), activation='relu')(up3)  # Adjust filter size as needed

concat3 = Concatenate()([up3, conv2])  # Shapes will now match (32,...)
conv11 = Conv2D(32, (3, 3), activation='relu', padding='same')(concat3)
conv12 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv11)

# Skip connections (consider adding based on noise complexity)
# ... add skip connections from corresponding encoder layers with matching shapes

# Output layer
denoised_outputs = Conv2D(N_CHANNELS, (1, 1), activation='sigmoid')(conv12)

# Build and compile the model
model = Model(inputs=noisy_inputs, outputs=denoised_outputs)
model.compile(loss='mse', optimizer='adam', metrics=['acccuracy'])  # Adjust loss and optimizer

# Train the model on your noisy images with corresponding clean images
# Implement data augmentation with noise variations and frequencies

# Evaluate the model on unseen noisy images and visualize results


model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 32  320         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 32  9248        ['conv2d[0][0]']                 
                                )                                                             

In [13]:
history = 0
with tf.device('/CPU:0'):
    history = model.fit(
        X_train, 
        y_train,
        epochs=3,
        callbacks=my_callbacks,
        # validation_data=(np.array(X_val), np.array(y_val)),
    )

Epoch 1/10
11/11 [==============================] - 129s 12s/step - loss: 23952.2480 - lr: 0.0010
Epoch 2/10
11/11 [==============================] - 128s 12s/step - loss: 23951.8398 - lr: 0.0010
Epoch 3/10
11/11 [==============================] - 154s 14s/step - loss: 23951.8398 - lr: 0.0010
Epoch 4/10
11/11 [==============================] - 129s 12s/step - loss: 23951.8379 - lr: 0.0010
Epoch 5/10
11/11 [==============================] - 130s 12s/step - loss: 23951.8398 - lr: 0.0010
Epoch 6/10
 1/11 [=>............................] - ETA: 2:08 - loss: 23729.1562

KeyboardInterrupt: 

In [15]:
model.evaluate(X_test)

3/3 [==============================] - 0s 3ms/step - loss: 0.0000e+00


0.0

In [16]:
pd.DataFrame(history.history)[['accuracy','val_accuracy']].plot()
plt.title("Accuracy")
plt.show()

pd.DataFrame(history.history)[['loss','val_loss']].plot()
plt.title("Loss")
plt.show()

AttributeError: 'int' object has no attribute 'history'

In [ ]:
predict = autoencoder.predict(X_test[0])
cv2.PSNR(predict, y_test[0])